In [1]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'GeForce MX150'

In [1]:
import gym
import math
import random
import numpy as np

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

In [2]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size